In [1]:
import SimpleITK as sitk
import numpy as np
import os

# Folder name where registration transforms are saved
folder = "TestDenoising5"

path = "./oai_zib_mri_train/train"
labelPath = "oai_zib_labelmaps/labelmaps/train"

P = [3, 3, 3]
N = [5, 5, 5]

In [2]:
# Read boundaries
file1 = open(f"./{folder}/boundaries.txt", "r")
temp = file1.read().split("\n")[:-1]
minx = int(temp[0])
maxx = int(temp[1])
miny = int(temp[2])
maxy = int(temp[3])
minz = int(temp[4])
maxz = int(temp[5])

In [3]:
file = open(f"./{folder}/names.txt", "r") 
name = file.read().split("\n")[0]

pathName = os.path.join(path, name)
images = []
images.append(sitk.GetArrayFromImage(sitk.ReadImage(pathName)))

shape = [[minx - (P[0]//2 + N[0]//2), maxx + (P[0]//2 + N[0]//2)],
         [miny - (P[1]//2 + N[1]//2), maxy + (P[1]//2 + N[1]//2)],
         [minz - (P[2]//2 + N[2]//2), maxz + (P[2]//2 + N[2]//2)],
        ]

copyShape = [[],[],[]]

# x range
if shape[0][0] < 0: 
    copyShape[0].append(0)
else:
    copyShape[0].append(shape[0][0])
    
if shape[0][1] > images[0].shape[0] - 1: 
    copyShape[0].append(images[0].shape[0] - 1)
else:
    copyShape[0].append(shape[0][1])

# y range
if shape[1][0] < 0: 
    copyShape[1].append(0)
else:
    copyShape[1].append(shape[1][0])
    
if shape[1][1] > images[0].shape[1] - 1: 
    copyShape[1].append(images[0].shape[1] - 1)
else:
    copyShape[1].append(shape[1][1])
    
#z range
if shape[2][0] < 0: 
    copyShape[2].append(0)
else:
    copyShape[2].append(shape[2][0])
    
if shape[2][1] > images[0].shape[2] - 1: 
    copyShape[2].append(images[0].shape[2] - 1)
else:
    copyShape[2].append(shape[2][1])
    
offset = []
offset.append(copyShape[0][0] - shape[0][0])
offset.append(copyShape[1][0] - shape[1][0])
offset.append(copyShape[2][0] - shape[2][0])

length = []
length.append(copyShape[0][1] - copyShape[0][0] + 1)
length.append(copyShape[1][1] - copyShape[1][0] + 1)
length.append(copyShape[2][1] - copyShape[2][0] + 1)

del images[0]
print(images)

[]


In [4]:
"""Read images and labels and make them numpy images with the proper size."""
from utils import *

images = []
labels = []

file = open(f"./{folder}/names.txt", "r") 
imageNames = file.read().split("\n")[:-1]
# del imageNames[1]

for i, name in enumerate(imageNames[:3]):
    print(f'Reading {name}. {i+1}/{len(imageNames)}')
    
    pathName = os.path.join(path, name)
    
    if i >= 1:
        t = sitk.ReadTransform(f"./{folder}/{name}.tfm")
        
        image = sitk.GetArrayFromImage(resampleImage(sitk.HistogramMatching(sitk.CurvatureFlow(sitk.ReadImage(pathName), 
                                                                                               timeStep = 0.04, 
                                                                                               numberOfIterations = 10), 
                                                                            images[0]), images[0], t))

        newImage = np.zeros((shape[0][1] - shape[0][0] + 1, shape[1][1] - shape[1][0] + 1, shape[2][1] - shape[2][0] + 1), dtype="uint16", order="F")
        newImage[offset[0]:offset[0]+length[0], 
                 offset[1]:offset[1]+length[1], 
                 offset[2]:offset[2]+length[2]] = image[copyShape[0][0]:copyShape[0][1]+1,
                                                        copyShape[1][0]:copyShape[1][1]+1,
                                                        copyShape[2][0]:copyShape[2][1]+1,]
        images.append(newImage)
        
        label = sitk.GetArrayFromImage(resampleSegmentation(sitk.ReadImage(os.path.join(labelPath, name)), images[0], t))
        
        newLabels = np.zeros((shape[0][1] - shape[0][0] + 1, shape[1][1] - shape[1][0] + 1, shape[2][1] - shape[2][0] + 1,), dtype="uint8", order="F")
        newLabels[offset[0]:offset[0]+length[0], 
                  offset[1]:offset[1]+length[1], 
                  offset[2]:offset[2]+length[2]] = label[copyShape[0][0]:copyShape[0][1]+1,
                                                         copyShape[1][0]:copyShape[1][1]+1,
                                                         copyShape[2][0]:copyShape[2][1]+1,]
        labels.append(newLabels)
    else:
        images.append(sitk.CurvatureFlow(sitk.ReadImage(pathName), timeStep = 0.04, numberOfIterations = 10))

        labels.append(sitk.ReadImage(os.path.join(labelPath, name)))
        
images[0] = sitk.GetArrayFromImage(images[0])
newImage = np.zeros((shape[0][1] - shape[0][0] + 1, shape[1][1] - shape[1][0] + 1, shape[2][1] - shape[2][0] + 1), dtype="uint16", order="F")
newImage[offset[0]:offset[0]+length[0], 
         offset[1]:offset[1]+length[1], 
         offset[2]:offset[2]+length[2]] = images[0][copyShape[0][0]:copyShape[0][1]+1,
                                                    copyShape[1][0]:copyShape[1][1]+1,
                                                    copyShape[2][0]:copyShape[2][1]+1,]
images[0] = newImage

labels[0] = sitk.GetArrayFromImage(labels[0])
newLabels = np.zeros((shape[0][1] - shape[0][0] + 1, shape[1][1] - shape[1][0] + 1, shape[2][1] - shape[2][0] + 1,), dtype="uint8", order="F")
newLabels[offset[0]:offset[0]+length[0], 
          offset[1]:offset[1]+length[1], 
          offset[2]:offset[2]+length[2]] = labels[0][copyShape[0][0]:copyShape[0][1]+1,
                                                     copyShape[1][0]:copyShape[1][1]+1,
                                                     copyShape[2][0]:copyShape[2][1]+1,]
labels[0] = newLabels

Reading 9005132.nii.gz. 1/60
Reading 9092628.nii.gz. 2/60
Reading 9131266.nii.gz. 3/60


In [20]:
from time import time
import spams
import math
# from sklearn.linear_model import Lasso

param = { 'K' : 10, 
          'lambda1' : 0.15,
          'iter' : 100}
b1 = 1

resDt = {}
resWt = {}

def applyDDL(segImage, images, labels, numOfLabels, P, N, b1=1.0, skipStep=1, lassoTol=0.01, lassoMaxIter=1e4, 
                      verboseX=True, verboseY=True, xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=-1, zmax=-1):  
    # Perform DDL
    size = segImage.shape
    assert (size == images[0].shape), "Segmentation images and images has different size."
    assert (size == labels[0].shape), "Segmentation images and labels has different size."
    assert (len(images) == len(labels)), "Images and labels number mismatch."
    assert (numOfLabels >= 1), "Number of labels lower than one."
    
    # Allocate memory for the segmentation
    newSegmentation = np.zeros(shape=size, dtype='uint8')
    
    # x range
    if not (xmin >= N[0]//2 + P[0]//2 and xmin < size[0] - N[0]//2 - P[0]//2 - 1):
        xmin = N[0]//2 + P[0]//2
    if not (xmax >= N[0]//2 + P[0]//2 and xmax < size[0] - N[0]//2 - P[0]//2 - 1 and xmax > xmin):
        xmax = size[0] - N[0]//2 - P[0]//2 - 1
        
    # y range
    if not (ymin >= N[1]//2 + P[1]//2 and ymin < size[1] - N[1]//2 - P[1]//2 - 1):
        ymin = N[1]//2 + P[1]//2
    if not (ymax >= N[1]//2 + P[1]//2 and ymax < size[1] - N[1]//2 - P[1]//2 - 1 and ymax > ymin):
        ymax = size[1] - N[1]//2 - P[1]//2 - 1
        
    # z range
    if not (zmin >= N[2]//2 + P[2]//2 and zmin < size[2] - N[2]//2 - P[2]//2 - 1):
        zmin = N[2]//2 + P[2]//2
    if not (zmax >= N[2]//2 + P[2]//2 and zmax < size[2] - N[2]//2 - P[2]//2 - 1 and zmax > zmin):
        zmax = size[2] - N[2]//2 - P[2]//2 - 1
    
    
    for x in range(xmin, xmax, skipStep):
        if verboseX:
            tstartX = time()
        for y in range(ymin, ymax, skipStep):
            if verboseY:
                print('\tX:', x, '\tY:', y , end=" ")
                tstartY = time()

            for z in range(zmin, zmax, skipStep):
                v = [x, y, z]

                L = createLDict(labels, v, N, numOfLabels)
                A = createA(images, v, P, N)
                B = createB(segImage, v, P)

                P_L = np.asfortranarray(np.concatenate((A, L*math.sqrt(b1)), axis=0), dtype = np.float64)

                D = spams.trainDL_Memory(P_L, **param)

                Dt = D[:A.shape[0],:]
                DtNorm = np.linalg.norm(Dt, axis=0)
                DtNorm[DtNorm == 0.] = 1
                # print(DtNorm)
                Dt /= DtNorm
                Wt = (D[A.shape[0]+1:,:] / math.sqrt(b1)) / DtNorm
                
                resDt[(x,y,z)] = Dt
                resWt[(x,y,z)] = Wt
                
                #################################
#                 alpha = spams.lasso(P_L,D = D, lambda1 = 0.15)
#                 xd = P_L - D * alpha
#                 print(P_L)
#                 print(xd)
#                 print(Wt)
#                 print()
                #################################

            if verboseY:
                print("\tTime:", time() - tstartY)
        if verboseX:
                print("X:", x, "\tTime:", time() - tstartX)
    return newSegmentation

In [21]:
segmentation = applyDDL(images[0], images[1:], labels[1:], 5, P, N, lassoTol=0.00001, lassoMaxIter=1e4, verboseX=True, 
                        verboseY=False, xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=60, zmax=61)

X: 3 	Time: 888.454621553421
X: 4 	Time: 972.752950668335


KeyboardInterrupt: 

NameError: name 'DtNorm' is not defined